# Find the Most Underpriced Flats (Side Project)

## Import Packages

In [1]:
# Web - Scraping and API Requests
import requests
from httpx import AsyncClient, Response
from parsel import Selector
import parsel
import jmespath
import asyncio

# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json
from typing import List
from typing import TypedDict

# Data Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# url displays
from IPython.display import display, Markdown

# Database Connection
from sqlalchemy import create_engine
from sqlalchemy import inspect, text

# File and System Operations
import os
import sys

## Other Setup

In [2]:
# This allows one to reload the custom package without having to install it again
%load_ext autoreload 

In [3]:
# this allows one to reload the custom package without having to install it again
%autoreload 1

sys.path.insert(0,'../src/')

# Import the custom package and sub-packages
%aimport rental_utils
%aimport rental_utils.functions
%aimport rental_utils.sql_queries

In [4]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

### Import Custom Packages

In [5]:
from rental_utils import sql_queries as sqlq
from rental_utils import functions as rent

## Analyse the Data

### Get the Data from the Database

In [6]:
engine = sqlq.get_sql_engine("../data/properties.db")

In [7]:
with engine.connect() as connection:
    properties_data = pd.read_sql(text(sqlq.GET_PROPERTIES_DATA_SQL_QUERY), connection)

### See which Flats are Underpriced

In [8]:


def find_underpriced(df, user_budget=1200):
    """Finds underpriced flats relative to others with the same travel time
    Takes as input a dataframe with the information, and the user's budget, and outputs a sorted 
    dataframe with the most underpriced rental properties at the top, and
    a recommendation with a link to the most ideal such property."""
    # Calculate the difference between predictions and the actual price
    df = df.copy()
    df.loc[:, 'savings'] = df['predicted_price_per_bed'] - df['price_per_bed']

    # Filter out the data for only that which is within the user's input budget
    budget_data = df[df['price_per_bed'] <= user_budget]

    # sort in descending order of 'savings' column: the most undervalued flat first
    sorted_data = budget_data.sort_values(by='savings', ascending=False)

    # Output the most underpriced flat
    if not sorted_data.empty:
        top_flat = sorted_data.iloc[0]
        address = top_flat['displayAddress']
        price = top_flat['price_per_bed']
        pred_price = top_flat['predicted_price_per_bed']
        url = top_flat['propertyUrl']
        full_url = f"https://www.rightmove.co.uk{url}" if url.startswith('/') else url
        display(Markdown(
            f"Your most underpriced flat is at **{address}** for a rent per bedroom of **£{price:.2f}**, while similar properties fetch **£{pred_price:.2f}**.<br>"
            f"[Click here to view the property]({full_url})"
        ))
    else:
        print("No flats found within your budget.")
    return sorted_data

sorted_data = find_underpriced(properties_data)
sorted_data.head()


Your most underpriced flat is at **Holloway Road, London, N7 8LZ** for a rent per bedroom of **£1125.00**, while similar properties fetch **£2405.24**.<br>[Click here to view the property](https://www.rightmove.co.uk/properties/163986716#/?channel=RES_LET)

,id,distance,travel_time,price_per_bed,predicted_price_per_bed,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription,savings
31,163986716,0,1350,1125.000000,2405.236695,4,1,8,"Holloway Road, London, N7 8LZ",51.549084,-0.107999,Flat,new,2025-06-30T19:47:03Z,4500,monthly,0,0,rent,0,94 sq. m.,/properties/163986716#/?channel=RES_LET,2025-06-30T19:41:53Z,Added today,4 bedroom flat,1280.236695
29,163986206,0,1562,1200.000000,2164.398705,2,1,9,"Bolingbroke Road, Olympia, London, W14",51.500493,-0.216429,Flat,new,2025-06-30T19:03:22Z,2400,monthly,0,0,rent,0,58 sq. m.,/properties/163986206#/?channel=RES_LET,2025-06-30T18:57:39Z,Added today,2 bedroom flat,964.398705
28,163986491,0,1752,1066.666667,1948.553337,3,1,8,"Tooting Bec Road, Tooting Bec, London, SW17 8BW",51.433952,-0.155108,Flat,new,2025-06-30T19:23:02Z,3200,monthly,0,0,rent,0,,/properties/163986491#/?channel=RES_LET,2025-06-30T19:17:54Z,Added today,3 bedroom flat,881.886670
8,163986725,0,1783,1075.000000,1913.336461,2,1,7,"Garfield Road, Battersea, London, SW11",51.464972,-0.152273,Flat,new,2025-06-30T19:50:03Z,2150,monthly,0,0,rent,0,46 sq. m.,/properties/163986725#/?channel=RES_LET,2025-06-30T19:44:36Z,Added today,2 bedroom flat,838.336461
25,163986278,0,1869,1050.000000,1815.638031,2,1,9,"Burgoyne Road, Harringey, N4",51.578216,-0.099920,Apartment,new,2025-06-30T19:05:02Z,2100,monthly,0,0,rent,0,,/properties/163986278#/?channel=RES_LET,2025-06-30T18:59:51Z,Added today,2 bedroom apartment,765.638031
